In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from mpl_toolkits.mplot3d import Axes3D

In [6]:
df = pd.read_csv('us_bank_wages/us_bank_wages.txt', sep="\t")
df.head()

,Unnamed: 0,SALARY,EDUC,SALBEGIN,GENDER,MINORITY,JOBCAT
0,0,57000,15,27000,1,0,3
1,1,40200,16,18750,1,0,1
2,2,21450,12,12000,0,0,1
3,3,21900,8,13200,0,0,1
4,4,45000,15,21000,1,0,1


In [7]:
df.shape

(474, 7)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 474 entries, 0 to 473
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Unnamed: 0  474 non-null    int64
 1   SALARY      474 non-null    int64
 2   EDUC        474 non-null    int64
 3   SALBEGIN    474 non-null    int64
 4   GENDER      474 non-null    int64
 5   MINORITY    474 non-null    int64
 6   JOBCAT      474 non-null    int64
dtypes: int64(7)
memory usage: 26.0 KB


In [9]:
df.describe()

,Unnamed: 0,SALARY,EDUC,SALBEGIN,GENDER,MINORITY,JOBCAT
count,474.000000,474.000000,474.000000,474.000000,474.000000,474.000000,474.000000
mean,236.500000,34419.567511,13.491561,17016.086498,0.544304,0.219409,1.411392
std,136.976275,17075.661465,2.884846,7870.638154,0.498559,0.414284,0.773201
min,0.000000,15750.000000,8.000000,9000.000000,0.000000,0.000000,1.000000
25%,118.250000,24000.000000,12.000000,12487.500000,0.000000,0.000000,1.000000
50%,236.500000,28875.000000,12.000000,15000.000000,1.000000,0.000000,1.000000
75%,354.750000,36937.500000,15.000000,17490.000000,1.000000,0.000000,1.000000
max,473.000000,135000.000000,21.000000,79980.000000,1.000000,1.000000,3.000000
